# Исследование надежности заемщиков.

**Цель исследования** — проверить четыре гипотезы:
1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
4. Как разные цели кредита влияют на его возврат в срок?


### Шаг 1. Обзор данных

In [1]:
#импортируем нужную для работы библиотеку
import pandas as pd

In [2]:
df = pd.read_csv('borrowers.csv')

In [3]:
#получаем общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита;

### Знакомство с данными

In [4]:
#выводим первые 10 строк таблицы
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [5]:
#знакомимся со столбцами таблицы
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Выводы**
1. В столбце days_employed(общий трудовой стаж) - отрицательные значения,что является ошибкой
2. В столбце education(уровень образования клиента) - присутствуют неявные дубликаты
3. В таблице названия столбцов корректны,написаны в правильном стиле,пропуски(пробелы) отсутствуют
4. В пятой строке(под индексом 4) присутствует аномальное значение,так как это значения указаны в днях,если перевести его в годы=938 лет,что является ошибкой

### Предобработка данных

In [6]:
#проверяем таблицу на пропущенные значения
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
#Исправим значения с отрицательных на положительные с помощью функции,которая возвращает абсолютное значение числа (модуль числа)
df['days_employed'] = df['days_employed'].abs()
#проверим
print(df['days_employed'].head())

0      8437.673028
1      4024.803754
2      5623.422610
3      4124.747207
4    340266.072047
Name: days_employed, dtype: float64


### Шаг 2.1 Заполнение пропусков и Изменение типов данных.

In [8]:
#заполним пропущенные значения медианой и приведем к целочисленному типу
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median()).astype(int)
#заполним пропуски медианой и приведем к целочисленному типу
df['total_income'] = df['total_income'].fillna(df['total_income'].median()).astype(int)

In [9]:
#пропущенные значения исправлены
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [10]:
#тип в столбце ежемесячные доходы исправлено
df.dtypes

children             int64
days_employed        int32
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int32
purpose             object
dtype: object

In [11]:
#уникальные значения столбца 
df['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

### Шаг 2.4. Удаление дубликатов.

In [12]:
df['education'] = df['education'].str.lower()
#проверяем
df['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [13]:
#уникальные значения столбца 
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

- В данном столбце приводим значения к одному стилю

In [14]:
#уникальные значения столбца 
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

- В данном столбце стиль выдержан, исправим 'ремонт жилью' на 'ремонт жилья'

In [15]:
df['purpose'] = df['purpose'].replace('ремонт жилью', 'ремонт жилья')
#проверим
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [16]:
#проверим таблицу на явные дубликаты и посчитаем их
df.duplicated().sum()

71

In [17]:
#удалим дубликаты,создадим новую индексацию и удалим старую
df = df.drop_duplicates().reset_index(drop=True)

In [18]:
#проверим
df.duplicated().sum()

0

### Шаг 2.2 Проверка данных на аномалии и исправления.
**Цель**
- Найти в данных такие значения,которые отклоняются и/или не соответствуют действительности


In [19]:
df.loc[df.loc[:, 'children'] == df['children'].max()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,880,21,среднее,1,женат / замужем,0,M,компаньон,0,145334,покупка жилья
720,20,855,44,среднее,1,женат / замужем,0,F,компаньон,0,112998,покупка недвижимости
1074,20,3310,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518,получение образования
2510,20,2714,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474,операции с коммерческой недвижимостью
2940,20,2161,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
20941,20,1240,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524,свой автомобиль
21255,20,601,37,среднее,1,женат / замужем,0,F,компаньон,0,102986,профильное образование
21320,20,2194,53,среднее,1,женат / замужем,0,M,компаньон,0,145017,покупка жилой недвижимости
21334,20,494,52,среднее,1,женат / замужем,0,M,компаньон,0,156629,операции со своей недвижимостью


- В столбце, присутствуют аномальные значения, так как аномальное значение является полностью случайным, т.е не зависит от значений других данных и по этим данным не возможно восстановить эти значения предлагаю изменить аномальные значения на 0.

In [20]:
df.loc[df['children'] >= 10, 'children'] = 0
#проверим это
df.loc[df.loc[:, 'children'] >= 10]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


- аномальные значения в столбце 'children' исправлены

- Согласно статье 63 ТК РФ официально работать можно с 16 лет. Значит для мужчин максимальный трудовой стаж в днях будет примерно равен 17885 дней или 49 лет, для женщин это 14235 дней или 39 лет. Предлагаю взять среднее и опираться на него, получается, что средний трудовой стаж для М и Ж = 16060 дней или 44 года

In [21]:
#посмотрим строки котрые больше или равны этому значению
df.loc[df.loc[:, 'days_employed'] >= 16060]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
18,0,400281,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823,на покупку подержанного автомобиля
24,1,338551,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью
25,0,363548,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112,покупка недвижимости
30,1,335581,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21434,0,338904,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439,сыграть свадьбу
21437,0,386497,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638,недвижимость
21438,0,362161,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029,операции с недвижимостью
21447,0,373995,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем


* 3450 строк с аномальным значением, исправим это, для каждому всех значений больше или равных среднему(16060) сделаем эти значения средним 

In [22]:
df.loc[df['days_employed'] >= 16060, 'days_employed'] = 16060
#проверим
df.loc[df.loc[:, 'days_employed'] >= 16060]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,16060,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
18,0,16060,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823,на покупку подержанного автомобиля
24,1,16060,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547,операции с коммерческой недвижимостью
25,0,16060,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112,покупка недвижимости
30,1,16060,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456,операции с коммерческой недвижимостью
...,...,...,...,...,...,...,...,...,...,...,...,...
21434,0,16060,53,среднее,1,гражданский брак,1,M,пенсионер,0,75439,сыграть свадьбу
21437,0,16060,62,среднее,1,женат / замужем,0,M,пенсионер,0,72638,недвижимость
21438,0,16060,59,высшее,0,женат / замужем,0,M,пенсионер,0,73029,операции с недвижимостью
21447,0,16060,59,среднее,1,женат / замужем,0,F,пенсионер,0,153864,сделка с автомобилем


**Вывод:**
- аномальные значения в столбце 'days_employed' исправлены

- Проверим столбец 'dob_years' на аномалии

In [23]:
df['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

- Все значения целочисленные, но в данных присутствует нулевое значения, посмотрим их

In [24]:
df.loc[df.loc[:, 'dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,16060,0,среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,2664,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,1872,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилья
578,0,16060,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,1158,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19770,0,2194,0,среднее,1,женат / замужем,0,F,сотрудник,0,145017,жилье
20397,0,16060,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193,покупка своего жилья
20512,0,16060,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788,недвижимость
21110,2,108,0,высшее,0,женат / замужем,0,M,компаньон,0,240702,строительство жилой недвижимости


- 101 строка содержащая аномальное значение, предлагаю это удалить, т.к данных значений не много

In [25]:
df = df.loc[df['dob_years'] != 0]
#проверим
df.loc[df.loc[:, 'dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


- аномальные значения в столбце 'dob_years' исправлены

In [26]:
#смотрим значения столбца 'education_id'
df['education_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [27]:
#смотрим значения столбца 'family_status'
df['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [28]:
#смотрим значения столбца 'family_status_id'
df['family_status_id'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [29]:
#смотрим значения столбца 'gender'
df['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

- необычное значение в столбце 'gender', изучим его

In [30]:
df.loc[df.loc[:, 'gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10684,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


**Вывод:**
- одна строка из 21000, предлагаю оставить её без изменений, так как она не повлияет на итоговые значения

In [31]:
#смотрим значения столбца 'income_type'
df['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [32]:
#смотрим значения столбца 'debt'
df['debt'].unique()

array([0, 1], dtype=int64)

In [33]:
#смотрим max, min значения столбца 'total_income'
df['total_income'].max(), df['total_income'].min()

(2265604, 20667)

- значения без аномалий

In [34]:
df['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Вывод**
 - В данных много аномальных значений, которые были исправлены, причины возникновения могут быть вызваны техническим или человеческим фактором, нужно разбираться 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

- Создадим два датафрейма:
1. Каждому уникальному значению из family_status соответствует уникальное значение family_status_id
2. Каждому уникальному значению из education соответствует уникальное значение education_id 

In [35]:
frame_family = df[['family_status', 'family_status_id']]
frame_family = frame_family.drop_duplicates().reset_index(drop=True)
frame_family

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


In [36]:
frame_education = df[['education', 'education_id']]
frame_education = frame_education.drop_duplicates().reset_index(drop=True)
frame_education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


- Из исходного датафрейма удалим столбцы education и family_status, оставим только их идентификаторы: education_id и family_status_id

In [37]:
df = df.drop(columns=['education', 'family_status'], axis=1) 
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,16060,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


### Шаг 2.6. Категоризация дохода.

- Создадим столбец total_income_category и отсортируем кредитополучателей по следующим категориям:
1. 0–30000 — 'E';
2. 30001–50000 — 'D';
3. 50001–200000 — 'C';
4. 200001–1000000 — 'B';
5. 1000001 и выше — 'A'


In [38]:
df['total_income_category'] = df['total_income'].apply(lambda x: 'E' if x<=30000 else 'D' 
                                                if (x>=30001) & (x<=50000) else 'C'
                                                if (x>=50001) & (x<=200000) else 'B' 
                                                if (x>=200001) & (x<=1000000) else 'A')

In [39]:
#посмотрим последние 10 строк
df.tail(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
21444,1,467,28,1,0,F,сотрудник,1,109486,заняться образованием,C
21445,0,914,42,0,0,F,компаньон,0,322807,покупка своего жилья,B
21446,0,404,42,0,1,F,компаньон,0,178059,на покупку своего автомобиля,C
21447,0,16060,59,1,0,F,пенсионер,0,153864,сделка с автомобилем,C
21448,1,2351,37,4,3,M,сотрудник,0,115949,покупка коммерческой недвижимости,C
21449,1,4529,43,1,1,F,компаньон,0,224791,операции с жильем,B
21450,0,16060,67,1,0,F,пенсионер,0,155999,сделка с автомобилем,C
21451,1,2113,38,1,1,M,сотрудник,1,89672,недвижимость,C
21452,3,3112,38,1,0,M,сотрудник,1,244093,на покупку своего автомобиля,B
21453,2,1984,40,1,0,F,сотрудник,0,82047,на покупку автомобиля,C


In [40]:
print('категория E: ' + str(len(df.loc[df.loc[:, 'total_income_category'] == 'E'])))
print('категория D: ' + str(len(df.loc[df.loc[:, 'total_income_category'] == 'D'])))
print('категория C: ' + str(len(df.loc[df.loc[:, 'total_income_category'] == 'C'])))
print('категория B: ' + str(len(df.loc[df.loc[:, 'total_income_category'] == 'B'])))
print('категория A: ' + str(len(df.loc[df.loc[:, 'total_income_category'] == 'A'])))

категория E: 22
категория D: 348
категория C: 15943
категория B: 5015
категория A: 25


**Вывод**
1. Меньше всего кредитополучателей в категории с самыми низкими и самыми высокими доходами
2. Больше всех в категории C, категория среднего класса

### Шаг 2.7. Категоризация целей кредита.
- На основание данных из столбца purpose сформируем новый столбец purpose_category, в который войдут следующие категории:
1. 'операции с автомобилем',
2. 'операции с недвижимостью',
3. 'проведение свадьбы',
4. 'получение образования'.
- Сделаем это для сокращения и более конкретного разделения на категории

In [41]:
auto = [
    'на покупку автомобиля', 'приобретение автомобиля', 'на покупку подержанного автомобиля', 
    'на покупку своего автомобиля', 'автомобили', 'сделка с подержанным автомобилем', 
    'автомобиль', 'свой автомобиль', 'сделка с автомобилем', 'на покупку автомобиля']

In [42]:
real_estate = [
    'покупка жилья', 'операции с жильем', 'покупка жилья для семьи', 'покупка недвижимости',
    'покупка коммерческой недвижимости', 'покупка жилой недвижимости', 'строительство собственной недвижимости',
    'недвижимость', 'строительство недвижимости', 'операции с коммерческой недвижимостью',
    'строительство жилой недвижимости', 'жилье', 'операции со своей недвижимостью', 'покупка своего жилья',
    'операции с недвижимостью', 'покупка жилья для сдачи', 'ремонт жилья']

In [43]:
wedding = ['сыграть свадьбу', 'на проведение свадьбы', 'свадьба']

In [44]:
education = [
    'дополнительное образование', 'образование', 'заняться образованием',
    'получение образования', 'получение дополнительного образования',
    'получение высшего образования', 'профильное образование',
    'высшее образование', 'заняться высшим образованием']

- Четыре списка, каждый содержит подкатегории из 4 основных категорий.

In [45]:
#Функция, которая будет категоризировать подкатегории
def categor(value):
    if value in auto: #если значения в списке(подкатегории auto) то функция вернет заданную категорию
        return 'операции с автомобилем' 
    if value in real_estate:
        return 'операции с недвижимостью'
    if value in wedding:
        return 'проведение свадьбы'
    if value in education:
        return 'получение образования' 

In [46]:
#Создадим новый столбец который будет содержать отобранные категории
#для этого к столбцу с помощью метода apply() применим нашу функцию
df['purpose_category'] = df['purpose'].apply(categor)

In [47]:
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,16060,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


- Функция выполнила свою задачу

In [48]:
#узнаем какие кредиты самые популярные, и посмотрим на них в порядке возрастания
df['purpose_category'].value_counts(ascending=True)

проведение свадьбы           2310
получение образования        3995
операции с автомобилем       4284
операции с недвижимостью    10764
Name: purpose_category, dtype: int64

- Как видим самый большой спрос на надвижимость,
- Второе место оставлю за  образованием и автомобилями,
- И меньше всего в категории свадьбы.

### Категоризация по количеству детей
- Предлагаю для удобства, распределить количество детей по следующим категориям:
1. Многодетные - от 1
2. Без детей - 0

In [49]:
def kids(value):
    if value == 0:
        return 'без детей'
    else:
        return 'многодетные'

In [50]:
df['children_type'] = df['children'].apply(kids)

In [51]:
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_type
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,многодетные
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,многодетные
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,без детей
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетные
4,0,16060,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,без детей


### Ответы на вопросы.
- **Есть ли зависимость между количеством детей и возвратом кредита в срок?**
- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**
- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**
- **Как разные цели кредита влияют на его возврат в срок?**

##### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [52]:
#сгруппируем данные в сводной таблице по столбцу debt, по значениям которые в колонке children_type,
#значения по которым мы хотим увидеть таблицу в колонке family_status_id и применим к ней функцию подсчета кол-ва вхождений
#символа в строку
data = df.pivot_table(index=['debt'], columns='children_type', values = 'family_status_id', aggfunc='count')

In [53]:
data

children_type,без детей,многодетные
debt,,
0,13031,6589
1,1066,667


In [54]:
data['без детей'][0]

13031

In [55]:
#вероятность просроченного кредита у многодетных
print(f'Многодетные: {data["многодетные"][1] / data["многодетные"][0]:.1%}')

print(f'Без детей: {data["без детей"][1] / data["без детей"][0]:.1%}')

Многодетные: 10.1%
Без детей: 8.2%


##### Вывод:

- Да, есть зависимость между количеством детей и возвратом кредита в срок
  если семья многодетная - от 1 ребенка, то вероятность задолжности увеличивается
- Разница между этими группами составляет 2%

##### Есть ли зависимость между семейным положением и возвратом кредита в срок?

- 1 вариант

In [56]:
data_subcategory = df.merge(frame_family, on='family_status_id', how='inner')

In [57]:
data_subcategory

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_type,family_status
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,многодетные,женат / замужем
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,многодетные,женат / замужем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,без детей,женат / замужем
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,многодетные,женат / замужем
4,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью,без детей,женат / замужем
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21348,0,8799,45,0,4,F,сотрудник,0,146945,ремонт жилья,C,операции с недвижимостью,без детей,Не женат / не замужем
21349,0,354,35,0,4,F,компаньон,0,128897,недвижимость,C,операции с недвижимостью,без детей,Не женат / не замужем
21350,0,578,26,2,4,M,компаньон,0,77813,операции с коммерческой недвижимостью,C,операции с недвижимостью,без детей,Не женат / не замужем
21351,0,165,26,0,4,M,компаньон,0,147301,получение дополнительного образования,C,получение образования,без детей,Не женат / не замужем


In [58]:
data = data_subcategory.pivot_table(index=['debt'], columns='family_status', values = 'family_status_id', aggfunc='count')

In [59]:
data

family_status,Не женат / не замужем,в разводе,вдовец / вдова,гражданский брак,женат / замужем
debt,,,,,
0,2521,1100,892,3744,11363
1,273,85,62,386,927


In [60]:
print(f'4- не женатые / не замужем: {data["Не женат / не замужем"][1] /data["Не женат / не замужем"][0]:.1%}')
print(f'3- в разводе: {data["в разводе"][1] /data["в разводе"][0]:.1%}')
print(f'2- вдовец / вдова: {data["вдовец / вдова"][1] /data["вдовец / вдова"][0]:.1%}')
print(f'1- гражданский брак: {data["гражданский брак"][1] /data["гражданский брак"][0]:.1%}')
print(f'0- женат / замужем: {data["женат / замужем"][1] /data["женат / замужем"][0]:.1%}')

4- не женатые / не замужем: 10.8%
3- в разводе: 7.7%
2- вдовец / вдова: 7.0%
1- гражданский брак: 10.3%
0- женат / замужем: 8.2%


##### Вывод:

1. Да,есть зависимость между семейным положением и возвратом кредита
2. не женатые / не замужем - больше всех склонны к задержке оплаты
3. вдовцы / вдовы - меньше всех склонны к задержке
4. лица состоящие в гражданском браке практически так же склонны к просрочке по платежу как и не женатые / не замужем
5. коментарий студента: можно ли объеденить гражданский брак + женат / замужем? Потому что брак и гражданский брак различие только в церковных обрядах, в шутку если то без участия церкви люди склонны к просрочке по кредиту, а если объеденить эти группы по факту бракосочетания то получится что женатые/замужние на 10+8-11=7% склонны к просрочке платежа чем не женатые / не замужем,


##### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [61]:
data = df.pivot_table(index=['debt'], columns='total_income_category', values ='total_income' , aggfunc='count')

In [62]:
data

total_income_category,A,B,C,D,E
debt,,,,,
0,23,4660,14590,327,20
1,2,355,1353,21,2


In [64]:
print(f'A: {data["A"][1]/data["A"][0]:.0%}')
print(f'B: {data["B"][1]/data["B"][0]:.0%}')
print(f'C: {data["C"][1]/data["C"][0]:.0%}')
print(f'D: {data["D"][1]/data["D"][0]:.0%}')
print(f'E: {data["E"][1]/data["E"][0]:.0%}')

A: 9%
B: 8%
C: 9%
D: 6%
E: 10%


1. 0–30000 — 'E';
2. 30001–50000 — 'D';
3. 50001–200000 — 'C';
4. 200001–1000000 — 'B';
5. 1000001 и выше — 'A

##### Вывод:
1. Наиболее склонны к просрочке по кредиту люди получающие до 30 т.р
2. Менее склонны к просрочке людие получающие среднюю зарплату, категория D
3. Удивительно, но люди получающие более 1млн. не менее склонны к задержке выплат, как и люди получающие до 30 т.р
4. Да,есть зависимость между уровнем дохода и задержке выплаты

##### Как разные цели кредита влияют на его возврат в срок?

In [65]:
data = df.pivot_table(index=['debt'], columns='purpose_category', values ='purpose' , aggfunc='count')

In [66]:
data

purpose_category,операции с автомобилем,операции с недвижимостью,получение образования,проведение свадьбы
debt,,,,
0,3884,9985,3625,2126
1,400,779,370,184


In [68]:
print(f'операции с автомобилем: {data["операции с автомобилем"][1] / data["операции с автомобилем"][0]:.0%}')
print(f'операции с недвижимостью: {data["операции с недвижимостью"][1] / data["операции с недвижимостью"][0]:.0%}')
print(f'получение образования: {data["получение образования"][1] / data["получение образования"][0]:.0%}')
print(f'проведение свадьбы: {data["проведение свадьбы"][1] / data["проведение свадьбы"][0]:.0%}')

операции с автомобилем: 10%
операции с недвижимостью: 8%
получение образования: 10%
проведение свадьбы: 9%


##### Вывод:
1. Наиболее склонны к задержке выплат люди использующие средства на автомобили и образование
2. Менее склонны к задержке люди берущие средства на недвижимость
3. На свадьбы люди практически так же склонны как и люди использующие средства на автомобили и образование

In [69]:
data = df.pivot_table(index=['purpose_category'], values ='debt')

In [70]:
data

,debt
purpose_category,
операции с автомобилем,0.093371
операции с недвижимостью,0.072371
получение образования,0.092616
проведение свадьбы,0.079654


In [71]:
print(round(data["debt"]*100, 2))

purpose_category
операции с автомобилем      9.34
операции с недвижимостью    7.24
получение образования       9.26
проведение свадьбы          7.97
Name: debt, dtype: float64


## Общий вывод:

1. В изученной таблицу очень много артефактов, нужно разбираться в причине появления
2. Использую сводные таблицы, мы ответили на поставленные вопросы, а именно:

1. Есть ли зависимость между количеством детей и возвратом кредита в срок?
- Да, наличие даже 1 ребенка влияет на возврат кредита в срок
2. Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Да, есть зависимость между семейным положением и возвратом кредита в срок,
  так менее всего склонны к задержке вдовцы/вдовы, наиболее склонны люди не состоящие в браке 
3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Да, есть зависимость между доходом и возвратом кредита,
  так менее всего склонны к задержке люди получающие среднюю зарплату от 30-50, более всего склонны получающие до 30т.р
4. Как разные цели кредита влияют на его возврат в срок?
- Да, разные цели влияют на возврат кредита в срок,
  к задержке более склонны люди берущие кредит на авто и образование,
  менее склонны берущие на недвижимость